In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE71799"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE71799"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE71799.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE71799.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE71799.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of molecular signatures of cystic fibrosis disease status using plasma-based functional genomics"
!Series_summary	"The complex milieu of inflammatory mediators associated with many diseases is often too dilute to directly measure in the periphery, necessitating development of more sensitive measurements suitable for mechanistic studies, earlier diagnosis, guiding selection of therapy, and monitoring interventions.  Previously, we determined that plasma of recent-onset (RO) Type 1 diabetes (T1D) patients induce a proinflammatory transcriptional signature in fresh peripheral blood mononuclear cells (PBMC) relative to that of unrelated healthy controls (HC).  Here, using an optimized cryopreserved PBMC-based protocol, we compared the signature found between unrelated healthy controls and non-diabetic cystic fibrosis patients possessing Pseudomonas aeruginosa pulmonary tract infection."
!Series_overall_design	"UPN727 cells were stimulat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any, Union

# Define data availability and conversion functions based on the provided information

# 1. Gene Expression Data Availability
# Based on the series summary and overall design, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the background information, we know this study compares cystic fibrosis patients
# with healthy controls, so the trait information is available even if not in sample characteristics
trait_row = None  # Not directly available in sample characteristics, but exists in the dataset
age_row = None    # Not mentioned in sample characteristics
gender_row = None  # Not mentioned in sample characteristics

# 2.2 Data Type Conversion
# Define conversion functions for trait, age, and gender
def convert_trait(value: str) -> Optional[int]:
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: CF=1, HC=0
    value = value.lower()
    if 'cf' in value or 'cystic fibrosis' in value:
        return 1
    elif 'hc' in value or 'healthy control' in value or 'control' in value:
        return 0
    return None

# Define placeholder conversion functions for age and gender
def convert_age(value: str) -> Optional[float]:
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: female=0, male=1
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Based on the background information, we know trait data is available
# even though it's not directly in the sample characteristics
is_trait_available = True

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None but we know trait data exists elsewhere in the dataset,
# we cannot use geo_select_clinical_features at this stage
# The extraction of trait information will need to be handled in later steps


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 74
Header line: "ID_REF"	"GSM1845886"	"GSM1845887"	"GSM1845888"	"GSM1845889"	"GSM1845890"	"GSM1845891"	"GSM1845892"	"GSM1845893"	"GSM1845894"	"GSM1845895"	"GSM1845896"	"GSM1845897"	"GSM1845898"	"GSM1845899"	"GSM1845900"	"GSM1845901"	"GSM1845902"	"GSM1845903"	"GSM1845904"	"GSM1845905"	"GSM1845906"	"GSM1845907"	"GSM1845908"	"GSM1845909"	"GSM1845910"	"GSM1845911"	"GSM1845912"	"GSM1845913"	"GSM1845914"	"GSM1845915"	"GSM1845916"	"GSM1845917"	"GSM1845918"	"GSM1845919"	"GSM1845920"	"GSM1845921"	"GSM1845922"	"GSM1845923"	"GSM1845924"	"GSM1845925"	"GSM1845926"	"GSM1845927"	"GSM1845928"	"GSM1845929"	"GSM1845930"	"GSM1845931"	"GSM1845932"	"GSM1845933"	"GSM1845934"	"GSM1845935"	"GSM1845936"	"GSM1845937"	"GSM1845938"	"GSM1845939"	"GSM1845940"	"GSM1845941"	"GSM1845942"	"GSM1845943"	"GSM1845944"	"GSM1845945"	"GSM1845946"	"GSM1845947"	"GSM1845948"	"GSM1845949"	"GSM1845950"	"GSM1845951"	"GSM1845952"	"GSM1845953"	"GSM1845954"	"GSM1845955"	"GSM1845956"	"GSM1845957"	"GSM1845958"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The first few identifiers shown are: '1007_s_at', '1053_at', '117_at', '121_at', etc.

# These identifiers appear to be Affymetrix probe IDs from a microarray platform,
# not standard human gene symbols like BRCA1, TP53, etc.
# Affymetrix IDs typically have this format with numbers followed by "_at" suffixes
# Therefore, they would need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for gene mapping
# From the gene annotation preview, 'ID' column contains probe IDs (matching gene expression data index)
# and 'Gene Symbol' contains the gene symbols we want to map to
probe_id_column = 'ID'  # Column containing probe IDs matching the gene expression data
gene_symbol_column = 'Gene Symbol'  # Column containing gene symbols

# 2. Get gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_id_column, gene_col=gene_symbol_column)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene expression data
print("Gene expression data after mapping (first 5 genes):")
print(gene_data.head(5))
print(f"Shape of gene expression data: {gene_data.shape}")


Gene expression data after mapping (first 5 genes):
          GSM1845886  GSM1845887  GSM1845888  GSM1845889  GSM1845890  \
Gene                                                                   
A1BG         4.89781     5.10467     4.84837     4.90231     5.02062   
A1BG-AS1     3.52356     4.29494     3.88388     4.12250     3.86527   
A1CF         5.75930     5.74441     5.66063     5.26587     5.30560   
A2M          5.27707     5.52795     5.21202     4.78926     5.44513   
A2M-AS1      4.92566     5.22810     4.97152     4.99641     4.62297   

          GSM1845891  GSM1845892  GSM1845893  GSM1845894  GSM1845895  ...  \
Gene                                                                  ...   
A1BG         4.96612     4.74118     5.29547     4.93891     4.85341  ...   
A1BG-AS1     3.96104     3.97486     4.19960     4.06677     3.68093  ...   
A1CF         5.67823     5.60140     5.79863     5.77698     5.88500  ...   
A2M          5.39855     5.40238     5.36047     5.36353  

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Create clinical features manually based on background information
# From Step 1 background information, we know the dataset compares:
# - unrelated healthy controls (uHC) plasma (n=31)
# - plasma from patients with cystic fibrosis (CF) with Pseudomonas infection (n=103)
# The total sample count matches our gene data columns (134)

# Get sample IDs from the gene data
sample_ids = normalized_gene_data.columns.tolist()

# Create a DataFrame with the trait information
# According to the background info: first 31 samples are healthy controls (0), remaining 103 are CF patients (1)
trait_name = "CF_Status"  # Using a clear column name rather than the variable 'trait'
cf_status = pd.Series([0] * 31 + [1] * 103, index=sample_ids, name=trait_name)
clinical_df = pd.DataFrame({trait_name: cf_status})

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link the clinical and genetic data
linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")
print("Linked data columns (first few):")
print(linked_data.columns[:10].tolist())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_name)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_name)

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data comparing cystic fibrosis patients with Pseudomonas infection to healthy controls."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 134)
First few genes with their expression values after normalization:
          GSM1845886  GSM1845887  GSM1845888  GSM1845889  GSM1845890  \
Gene                                                                   
A1BG         4.89781     5.10467     4.84837     4.90231     5.02062   
A1BG-AS1     3.52356     4.29494     3.88388     4.12250     3.86527   
A1CF         5.75930     5.74441     5.66063     5.26587     5.30560   
A2M          5.27707     5.52795     5.21202     4.78926     5.44513   
A2M-AS1      4.92566     5.22810     4.97152     4.99641     4.62297   

          GSM1845891  GSM1845892  GSM1845893  GSM1845894  GSM1845895  ...  \
Gene                                                                  ...   
A1BG         4.96612     4.74118     5.29547     4.93891     4.85341  ...   
A1BG-AS1     3.96104     3.97486     4.19960     4.06677     3.68093  ...   
A1CF         5.67823     5.60140     5.79863     5.77698     5.88500  ...   
A2M

Normalized gene data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE71799.csv
Clinical data saved to ../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE71799.csv
Clinical data preview:
{'CF_Status': [0, 0, 0, 0, 0]}
Linked data shape (before handling missing values): (134, 19846)
Linked data columns (first few):
['CF_Status', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT']


Data after handling missing values: (134, 19846)
For the feature 'CF_Status', the least common label is '0' with 31 occurrences. This represents 23.13% of the dataset.
The distribution of the feature 'CF_Status' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cystic_Fibrosis/GSE71799.csv
